In [13]:
import asyncio
import websockets
import json
from datetime import datetime
from pymongo import MongoClient
import paho.mqtt.client as mqtt

# Configuration MongoDB
mongo_client = MongoClient('mongodb://localhost:27017/')
db_name = "hopital"

# Configuration MQTT
mqtt_broker = "127.0.0.1"
mqtt_port = 1883

# Liste des clients WebSocket connectés
connected_clients = set()

# Callback pour la connexion MQTT
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected to MQTT broker")
    else:
        print(f"Failed to connect to MQTT broker with result code {rc}")

    client.subscribe("vital_signs/+/temperature", qos=1)
    client.subscribe("vital_signs/+/heart_rate", qos=1)
    client.subscribe("vital_signs/+/spo2", qos=1)
    client.subscribe("vital_signs/+/breath_rate", qos=1)
    client.subscribe("vital_signs/+/ecg", qos=2)

# Callback pour la réception de messages MQTT
def on_message(client, userdata, msg):
    print(f"Received `{msg.payload.decode()}` from `{msg.topic}` topic")
    
    # Envoyer les données à tous les clients WebSocket connectés
    data = {
        "topic": msg.topic,
        "value": msg.payload.decode(),
        "timestamp": datetime.now().isoformat()
    }
    asyncio.create_task(send_to_clients(data))
    
    # Enregistrer les données dans MongoDB
    topic_parts = msg.topic.split('/')
    bed_number_str = topic_parts[1]
    collection_name = topic_parts[2]

    bed_number = int(bed_number_str.replace('bed', ''))

    # Stocker les données dans la collection correspondante
    db = mongo_client[db_name]
    collection = db[collection_name]
    data = {
        "bed": bed_number,
        "value": msg.payload.decode(),
        "timestamp": datetime.now()
    }
    try:
        collection.insert_one(data)
        print(f"Data inserted into collection '{collection_name}' in database '{db_name}'")
    except Exception as e:
        print(f"Error inserting data into MongoDB: {e}")

# Fonction pour envoyer des données à tous les clients WebSocket
async def send_to_clients(data):
    if connected_clients:
        message = json.dumps(data)
        await asyncio.wait([client.send(message) for client in connected_clients])

# Gestion des connexions WebSocket
async def websocket_handler(websocket, path):
    # Ajouter le client à la liste des clients connectés
    connected_clients.add(websocket)
    try:
        await asyncio.sleep(3600)  # Garder le gestionnaire actif
    finally:
        # Retirer le client de la liste des clients connectés
        connected_clients.remove(websocket)

# Configuration du client MQTT
mqtt_client = mqtt.Client()
mqtt_client.on_connect = on_connect
mqtt_client.on_message = on_message

mqtt_client.connect(mqtt_broker, mqtt_port, 60)
mqtt_client.loop_start()

# Lancer le serveur WebSocket
async def main():
    await websockets.serve(websocket_handler, "0.0.0.0", 3003)

# Fonction principale avec asyncio
def run_server():
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(main())

if __name__ == "__main__":
    run_server()


Connected to MQTT broker


C:\Users\dell\AppData\Local\Temp\ipykernel_18092\1254255285.py:82: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  mqtt_client = mqtt.Client()


RuntimeError: Cannot run the event loop while another loop is running

In [9]:
asyncio.all_tasks()

{<Task pending name='Task-3' coro=<Kernel.dispatch_queue() running at c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel\kernelbase.py:542> cb=[IOLoop.add_future.<locals>.<lambda>() at c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\tornado\ioloop.py:688]>}

In [11]:
loop.stop('Task-3')

NameError: name 'loop' is not defined